# QUBO formulation of the Redundant Calibration

In [ ]:
import numpy as np

In [ ]:
A = np.random.rand(8,8)
A = 0.5*(A+A.T)

In [ ]:
b = 0.1 * np.random.rand(8,1)
b /= np.linalg.norm(b)

## Classical Solution

The solution of such a small system can be obtained by a least square as implemented in numpy

In [ ]:
npsol = np.linalg.solve(A,b)
npsol = np.asarray(npsol).flatten()

## 2. QUBO formalism for linear systems

The Quandratic Unconstrainted Binary Optimization problem, or QUBO, allows to minimize the cost function :

$$
E(x) = x^{T}Qx
$$

where the variables $x_i$ are binaries, i.e. the are 0 or 1. The equation above can be rewritten as :

$$
E(x) = \sum_i Q_{ii}x_i + \sum_{ij} Q_{ij}x_ix_j
$$

that is very similar to the Ising model, basis of the quantum annealler architecture. 

### Encoding real numbers in binary variables

In the QUBO problems, variables are binaries and we of course want to solve for real numbers in our case. There ar e different ways to encode real numbers in multiple binaries. In our case since the variables are between -1.0 and 1.0 we can use the following encoding : 

$$
r_i = a \sum_n x_n 2^{n} - x_{k+n} 2^{n} 
$$

where $a$ is a normalization constant. THis encoding is created in the `SolutionVector` class that allows to encode/decode real numbers in a series of binaries variables. We use here the `RealUnitQbitEncoding` to obtain real numbers between -1 and 1. The number of qbit controls the precision of the reals we can obtain.

### Linear systems

To solve a linear system $Ax=b$ with QUBO we need to minimize the following loss 

$$
E(x) = ||Ax-b||^2 =(Ax-b)^T (Ax-b) = x^T A^T A x-b^T A x-A x b^T+ b^T b = x^T Q x + ||b||^2
$$

We can ignore the last terms as it doesn't contribute to the optimization process. The $Q$ matrix can be defined through the different terms of the expression. This is achieved by the `create_qubo_matrix` methods that returns a `dict` containing the weights (diagonal terms) and strengths (off diagonal terms) of the $Q$ matrix. 

## 3. Solving the system

We will use here the `SimulatedAnnealingSampler` to be able to run that code locally. Quantum solvers are available through the Leap cloud service.

In [ ]:
from qalcore.dwave.qubols.qubols import QUBOLS
qubols = QUBOLS(A,b)
sol_num = qubols.solve(num_reads=1000)

In [ ]:
print(sol_num)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(npsol, sol_num)
plt.plot([-1,1],[-1,1],'--',c='gray')